<h1> Capstone Project </h1>
<p> RESTful API calls to the Foursquare API to retrieve data about venues in different neighborhoods around the world. 
Use of python and its pandas library to manipulate data. </p>

In [3]:
import numpy as np  
import pandas as pd

In [ ]:
print("Hello Capstone Project Course!")